<img src="slides/8_30.png">

In [1]:
import pandas as pd
import numpy as np
loan = pd.read_csv('data/loan_data.csv')

# 使用しない列を削除
df = loan.drop(['Unnamed: 0', 'status', 'loan_amnt', 'term', 'annual_inc', 'dti',
                  'revol_bal', 'revol_util', 'purpose', 'home_ownership',
                  'delinq_2yrs_zero', 'pub_rec_zero', 'open_acc', 'grade'], axis=1)

# ダミー変数化
df['outcome'] = df['outcome'].map(lambda x: 1 if x == 'default' else 0)
df = pd.get_dummies(data=df)

# 説明変数、目的変数を作成
y = 'outcome'
X = df.drop([y], axis=1)
Y = df[y]

### ホールドアウトにより予測精度を検証

```train_test_split()``` によりデータセットを分割

- ```test_size=0.3``` 30% をテストデータとして分割
- ```X_train, Y_train``` 教師データ
- ```X_test, Y_test``` テストデータ

```model.fit(X_train, Y_train)```

- 教師データで決定木を作成

```Y_predict = model.predict(X_test)```<br>
```Y_proba = model.predict_proba(X_test)```

- テストデータで目的変数を予測

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split

def calc_auc(model, X, Y):
    """
    正確度、適合度、再現率（感度）、AUCの表示
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

def classify(depth):
    # データを分割
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
    # 決定木を作成
    model = DecisionTreeClassifier(criterion='entropy', max_depth=depth)
    model.fit(X_train, Y_train)

    print('depth={:2}:\t{:.3f}\t\t{:.3f}'.format(
        depth, calc_auc(model, X_train, Y_train), calc_auc(model, X_test, Y_test)))

### 決定木の深さの数値を変更して実行

2 から 16 未満まで値を変更しながら、決定木を作成し、AUCを確認

In [3]:
print('AUC\t\t教師データ\tテストデータ')
for d in range(2, 16, 1):
    classify(d)

AUC		教師データ	テストデータ
depth= 2:	0.648		0.642
depth= 3:	0.669		0.664
depth= 4:	0.676		0.672
depth= 5:	0.684		0.674
depth= 6:	0.688		0.683
depth= 7:	0.698		0.673
depth= 8:	0.706		0.673
depth= 9:	0.716		0.666
depth=10:	0.724		0.663
depth=11:	0.734		0.666
depth=12:	0.760		0.645
depth=13:	0.773		0.644
depth=14:	0.793		0.635
depth=15:	0.803		0.633


教師データ
- 決定木が深くなるとAUCは大きくなる

テストデータ
- max_depth=5~8 あたりでAUCは最大化
- max_depth=10 以上でAUCは低下

データセットの分割はランダムに行われるため常に同じ結果は出ない

In [ ]:
# 実行のたびに結果が変わる（場合がほとんど）
classify(5)

<hr>

<img src="slides/8_31.png">